## Lede Algorithms 2018 Week 2 class 1 - Text analysis

Before running this notebook you will need to install a few things:

```
pip3 install textblob
python3 -m textblob.download_corpora
pip3 install scipy
pip3 install scikit-learn
```

For more text analysis goodness, check out Jonathan Soma's 2017 notebooks:
- [TextBlob spaCy sklearn lemmas stems and vectorization](http://jonathansoma.com/lede/algorithms-2017/classes/text-analysis/textblob-spacy-sklearn-lemmas-stems-and-vectorization/)
- [Counting and Stemming](http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/)


In [172]:
# Import the packages we will be using
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import math


Load some press releases to play around with. These were scraped from NJ Senator Menendez' site in 2012.

In [16]:
pr = pd.read_csv('menendez-press-releases.csv')
len(pr)

1530

In [223]:
pr.head()

,url,text
0,http://menendez.senate.gov/newsroom/press/rele...,Menendez Statement on Black History Month\n ...
1,http://menendez.senate.gov/newsroom/press/rele...,Menendez Praises Susan G. Komen For Reversing ...
2,http://menendez.senate.gov/newsroom/press/rele...,Menendez Applauds Dentists’ Pro-Bono Work For ...
3,http://menendez.senate.gov/newsroom/press/rele...,Senator Menendez Applauds Passage of STOCK Act...
4,http://menendez.senate.gov/newsroom/press/rele...,Menendez Hails Banking Committee Passage of Ir...


In [ ]:
These press releases are on all sorts of topics. Take a look at a few, like this:

In [346]:
print(pr.text[8])

Senator Menendez Slams Unfair Imprisonment of Former Ukrainian Prime Minister Yulia Tymoshenko
                    
                      February 1, 2012
                     WASHINGTON – United States Senator Robert Menendez (D-NJ) participated in the Senate Foreign Relations Committee hearing on Ukraine today, giving him the opportunity to meet Eugenia Tymoshenko and to discuss the inhumane detention of her mother, the former Prime Minister Yulia Tymoshenko.  Last October, a Ukrainian court sentenced Yulia Tymoshenko to seven years in prison after she was found guilty of abuse of office when brokering a 2009 gas deal with Russia.  The Senator expressed his sympathy and support to Ms. Tymoshenko and vowed to assist in her efforts to have her mother freed from prison.

 “Your mother is a pioneering and incredibly strong woman,” the Senator told the younger Tymoshenko. “Yulia is an example for all people who care so much about their country that they are willing to endure extraordinary

There are press releases about government programs, holidays, foreign policy, and more. Can an algorithm tell us something about which topics there are?


### Sentences and tokens

The first step of text analysis is typically breaking the text into sentences and words or more accurately, "tokens" which are basically words but can also be punctuation and numbers.

We'll use the TextBlob package, which has many easy and useful text processing methods -- though as we will see it's also kind of dumb in many cases.

Let's start by trying analyze the sentences of the first press release in the set.

In [337]:
press_release_text = pr.text[212]
doc = TextBlob(press_release_text)
doc.sentences

[Sentence("Menendez and Lautenberg Applaud USDOT’s $4.3 Million Award to National Transit Institute at Rutgers
                     
                             Funding will provide job training and education for public transportation workers
                     
                       August 3, 2011
                      WASHINGTON, D.C. – Today, U.S."),
 Sentence("Senators Robert Menendez (D-NJ) and Frank R. Lautenberg (D-NJ) applauded Secretary Ray LaHood on a $4.3 million grant from the U.S. Department of Transportation in support of the National Transit Institute (NTI) at Rutgers."),
 Sentence("Since 1991, NTI has served as the premiere research, training, and educational institute in the country dedicated to public transportation."),
 Sentence("This award will enable critical enhancements to NTI’s ongoing work, including safety and security training, procurement, planning and advanced technologies."),
 Sentence("“Transit is a critical element of our transportation network and r

As you can see, TextBlob is a little naive about what counts as a sentence. Or perhaps the problem is that real text contains lots of things that aren't really sentences. What should we do with the title and the dateline? Even so, it seems to have problems with quotes and newlines.

Anyway, let's take one of these sentences and play with it further. 

In [292]:
s = doc.sentences[9]
s

Sentence("“This grant will ensure that the existing and importantly, the new generation of public transportation workers receive the training that they need to comply with federal regulations and operate safe and efficient transit services,” said NTI’s director, Paul Larrousse.NTI works cooperatively through partnerships with industry, government, institutions, and associations to develop high quality programs that build careers and help make our communities healthier and more livable.")

This `Sentence` object acts just like a Python string. Let's try to break it into words for further analysis.

In [293]:
s.split(' ')

WordList(['“This', 'grant', 'will', 'ensure', 'that', 'the', 'existing', 'and', 'importantly,', 'the', 'new', 'generation', 'of', 'public', 'transportation', 'workers', 'receive', 'the', 'training', 'that', 'they', 'need', 'to', 'comply', 'with', 'federal', 'regulations', 'and', 'operate', 'safe', 'and', 'efficient', 'transit', 'services,”', 'said', 'NTI’s', 'director,', 'Paul', 'Larrousse.NTI', 'works', 'cooperatively', 'through', 'partnerships', 'with', 'industry,', 'government,', 'institutions,', 'and', 'associations', 'to', 'develop', 'high', 'quality', 'programs', 'that', 'build', 'careers', 'and', 'help', 'make', 'our', 'communities', 'healthier', 'and', 'more', 'livable.'])

Notice that all of the punctuation and capitalization is still there. If we're counting occurences of the word "nation" we will miss "Nation’s". We need a smarter way to extract words. This process is called tokenization.

In [294]:
s.tokens

WordList(['“', 'This', 'grant', 'will', 'ensure', 'that', 'the', 'existing', 'and', 'importantly', ',', 'the', 'new', 'generation', 'of', 'public', 'transportation', 'workers', 'receive', 'the', 'training', 'that', 'they', 'need', 'to', 'comply', 'with', 'federal', 'regulations', 'and', 'operate', 'safe', 'and', 'efficient', 'transit', 'services', ',', '”', 'said', 'NTI', '’', 's', 'director', ',', 'Paul', 'Larrousse.NTI', 'works', 'cooperatively', 'through', 'partnerships', 'with', 'industry', ',', 'government', ',', 'institutions', ',', 'and', 'associations', 'to', 'develop', 'high', 'quality', 'programs', 'that', 'build', 'careers', 'and', 'help', 'make', 'our', 'communities', 'healthier', 'and', 'more', 'livable', '.'])

But what about different forms of the same word? Suppose we want to count "education" and "educate" as the same thing? This is where _lemmatization_ and _stemming_ come in. 

In [295]:
for word in s.words:
    print("ORIGINAL:", word, "| LEMMA:", word.lemmatize(), "| STEM:", word.stem())

ORIGINAL: “ | LEMMA: “ | STEM: “
ORIGINAL: This | LEMMA: This | STEM: thi
ORIGINAL: grant | LEMMA: grant | STEM: grant
ORIGINAL: will | LEMMA: will | STEM: will
ORIGINAL: ensure | LEMMA: ensure | STEM: ensur
ORIGINAL: that | LEMMA: that | STEM: that
ORIGINAL: the | LEMMA: the | STEM: the
ORIGINAL: existing | LEMMA: existing | STEM: exist
ORIGINAL: and | LEMMA: and | STEM: and
ORIGINAL: importantly | LEMMA: importantly | STEM: importantli
ORIGINAL: the | LEMMA: the | STEM: the
ORIGINAL: new | LEMMA: new | STEM: new
ORIGINAL: generation | LEMMA: generation | STEM: gener
ORIGINAL: of | LEMMA: of | STEM: of
ORIGINAL: public | LEMMA: public | STEM: public
ORIGINAL: transportation | LEMMA: transportation | STEM: transport
ORIGINAL: workers | LEMMA: worker | STEM: worker
ORIGINAL: receive | LEMMA: receive | STEM: receiv
ORIGINAL: the | LEMMA: the | STEM: the
ORIGINAL: training | LEMMA: training | STEM: train
ORIGINAL: that | LEMMA: that | STEM: that
ORIGINAL: they | LEMMA: they | STEM: they
O

In this example `lemmatize()` mostly just removes pluralization. `stem()` goes further and produces many non-words like "develop". It's just a set of rules that try to parse out English morphology. The rule set here is called [Porter stemming](https://snowballstem.org/algorithms/porter/stemmer.html), and there are similar algorithms for [many languages](https://snowballstem.org/algorithms/). Lemmatization is actually smarter because it uses a dictionary, so it can undo common verb inflections... if you tell TextBlob that the word is a verb (to be fair, it can [figure out parts of speech](https://textblob.readthedocs.io/en/dev/quickstart.html#part-of-speech-tagging) if you ask it to.)

In [64]:
TextBlob("running").words[0].lemmatize('v')

'run'

In [65]:
TextBlob("ran").words[0].lemmatize('v')

'run'

For our purposes we're going to use TexBlob to make a basic tokenizing function that just makes everything lowercase and throws token with less than 3 characters, which throws out punctuation tokens too.

In [215]:
def tokenize(s):
    blob = TextBlob(s.lower())
    words = [token for token in blob.words if len(token)>2]
    return words

### Document similarity
One of the main things we want to do with documents is compare them to each other. This is useful for many things:
- matching search queries against documents
- classifying documents (think of this as sorting them into piles)
- clustering documents by topic

To do this, we need to write a function that takes two documents and returns a number representing some concept of "similarity" that will be useful for these tasks. To do _that_, we're first going to turn each document into a vector. Our first attempt at this will be by counting the number of tokens of each kind.

In [182]:
def doc2vec_count(s):
    tokens = tokenize(s)
    vec = {}
    for t in tokens:
        vec[t] = vec.get(t, 0) + 1
    return vec

In [175]:
doc2vec_count("the cat and the mat")

{'and': 1, 'cat': 1, 'mat': 1, 'the': 2}

The simplest way to compare two word count vectors is to count the number of overlapping words. Each word can appear more than once, so we'll multiply together the counts of the same word in each docmument. Why? Because this leads us to a Euclidian feature vector with natural geometric properties, which makes it possible to think about wbat is going on with spatial analogies. 

In [183]:
def doc_similarity(a_vec,b_vec):
    total = 0
    for word in a_vec:
        if word in b_vec:
            total += a_vec[word]*b_vec[word]
    return total

In [315]:
a = doc2vec_count(str(doc.sentences[6]))  # need str to convert Sentence object to string
b = doc2vec_count(str(doc.sentences[9]))

In [316]:
print(a)

{'nti': 1, 'work': 1, 'essential': 1, 'for': 1, 'the': 1, 'industry': 1, 'continue': 1, 'create': 1, 'good': 1, 'long': 1, 'term': 1, 'jobs': 2, 'provide': 1, 'families': 1, 'with': 1, 'access': 1, 'opportunity': 1, 'and': 6, 'help': 1, 'our': 1, 'communities': 1, 'grow': 1, 'ways': 1, 'that': 1, 'are': 1, 'smart': 1, 'efficient': 2, 'millions': 1, 'people': 1, 'count': 1, 'reliable': 1, 'public': 1, 'transportation': 1, 'get': 1, 'from': 1, 'their': 2, 'homes': 1, 'transit': 1, 'workers': 1, 'make': 1, 'this': 1, 'possible': 1, 'said': 1, 'senator': 1, 'lautenberg': 1}


In [317]:
print(b)

{'this': 1, 'grant': 1, 'will': 1, 'ensure': 1, 'that': 3, 'the': 3, 'existing': 1, 'and': 6, 'importantly': 1, 'new': 1, 'generation': 1, 'public': 1, 'transportation': 1, 'workers': 1, 'receive': 1, 'training': 1, 'they': 1, 'need': 1, 'comply': 1, 'with': 2, 'federal': 1, 'regulations': 1, 'operate': 1, 'safe': 1, 'efficient': 1, 'transit': 1, 'services': 1, 'said': 1, 'nti': 1, 'director': 1, 'paul': 1, 'larrousse.nti': 1, 'works': 1, 'cooperatively': 1, 'through': 1, 'partnerships': 1, 'industry': 1, 'government': 1, 'institutions': 1, 'associations': 1, 'develop': 1, 'high': 1, 'quality': 1, 'programs': 1, 'build': 1, 'careers': 1, 'help': 1, 'make': 1, 'our': 1, 'communities': 1, 'healthier': 1, 'more': 1, 'livable': 1}


In [318]:
doc_similarity(a,b)

58

Ok, but what does "58" mean? One problem we are going to have is that longer documents will tend to be more similar to everything else. More words mean more words can match. We will solve this problem by normalizing each document vector so that it has length 1, meaning that the sum of the _squares_ of the elements is one -- this is Pyhagoras, so we can think of a document as a unit vector now, or a direction, in a space that has as many dimensions as the vocabulary size. 


In [211]:
def doc2vec_normalized(s):
    tokens = tokenize(s)
    vec = {}
    for t in tokens:
        vec[t] = vec.get(t, 0) + 1 # get from dict with a default of 0 if missing
        
    length = math.sqrt(sum([x*x for x in vec.values()]))  # length of a vector, according to Pythagoras
    for word,value in vec.items():
        vec[word] /= length
        
    return vec

In [322]:
a = doc2vec_normalized(str(doc.sentences[6]))  # need str to convert Sentence object to string
b = doc2vec_normalized(str(doc.sentences[9]))
c = doc2vec_normalized(str(doc.sentences[5]))

In [323]:
print(c)

{'today': 0.1690308509457033, 'with': 0.3380617018914066, 'gas': 0.1690308509457033, 'prices': 0.1690308509457033, 'around': 0.1690308509457033, 'gallon': 0.1690308509457033, 'and': 0.3380617018914066, 'oil': 0.1690308509457033, 'companies': 0.1690308509457033, 'reaping': 0.1690308509457033, 'record': 0.1690308509457033, 'profits': 0.1690308509457033, 'the': 0.3380617018914066, 'threat': 0.1690308509457033, 'climate': 0.1690308509457033, 'change': 0.1690308509457033, 'growing': 0.1690308509457033, 'wealth': 0.1690308509457033, 'disparity': 0.1690308509457033, 'transit': 0.1690308509457033, 'part': 0.1690308509457033, 'solution': 0.1690308509457033, 'for': 0.1690308509457033, 'number': 0.1690308509457033, 'interconnected': 0.1690308509457033, 'challenges': 0.1690308509457033}


Now our similarity function says that the first two sentences are the most similar, because they have words like "industry", "efficient", and "transit" in common.

In [324]:
print(doc_similarity(a,b))
print(doc_similarity(b,c))
print(doc_similarity(a,c))

0.5943484047696529
0.37583907018239515
0.32251021858460976


Another way to look at document vectors is as list of top words. Let's sort document vectors by decreasing value to try to get an idea of what the entire press release document is "about", and print the top 20

In [249]:
def print_sorted_vector(v):
    # this "lambda" thing is an anonymous function, google me to unluck bonus coding knowledge
    sorted_list = sorted(v.items(), key=lambda x: (x[1],x[0]), reverse=True) 
    sorted_list = sorted_list[:20]
    print('\n'.join([str(x) for x in sorted_list]))

In [325]:
print_sorted_vector(a)

('and', 0.6359987280038161)
('their', 0.211999576001272)
('jobs', 0.211999576001272)
('efficient', 0.211999576001272)
('workers', 0.105999788000636)
('work', 0.105999788000636)
('with', 0.105999788000636)
('ways', 0.105999788000636)
('transportation', 0.105999788000636)
('transit', 0.105999788000636)
('this', 0.105999788000636)
('the', 0.105999788000636)
('that', 0.105999788000636)
('term', 0.105999788000636)
('smart', 0.105999788000636)
('senator', 0.105999788000636)
('said', 0.105999788000636)
('reliable', 0.105999788000636)
('public', 0.105999788000636)
('provide', 0.105999788000636)


In [326]:
print_sorted_vector(doc2vec_normalized(press_release_text))

('and', 0.6364382128493583)
('the', 0.35972594639311556)
('transportation', 0.19369858651936991)
('transit', 0.19369858651936991)
('nti', 0.19369858651936991)
('with', 0.13835613322812138)
('training', 0.13835613322812138)
('that', 0.13835613322812138)
('public', 0.13835613322812138)
('this', 0.1106849065824971)
('our', 0.1106849065824971)
('for', 0.1106849065824971)
('workers', 0.08301367993687282)
('work', 0.08301367993687282)
('will', 0.08301367993687282)
('said', 0.08301367993687282)
('rutgers', 0.08301367993687282)
('menendez', 0.08301367993687282)
('lautenberg', 0.08301367993687282)
('institute', 0.08301367993687282)


Not too bad... but is this press release really "about" the words like "the" and "and"? We need something better.

### TF-IDF weighting
As we discussed in class, term frequency / inverse document frequency is a word weighting scheme that tries to give less weight to words that appear in many documents. This will solve our "the" problem, and it will also help drop out topic words that are common to the entire corpus. Rather than writing it ourselves, we're going to use the implementation in the `scikit` library.

Scikit includes a bunch of built in vectorizers, such as classic counting. Let's turn the first ten press releases into vectors.

In [208]:
# Make a new Count Vectorizer!!!! It will conveniently remove stop words if we tell it what language we're using
vectorizer = CountVectorizer(stop_words='english', tokenizer=tokenize)

# Use the vectorizor we just made. The name fit_transform will be clearer later when we use it for machine learning
matrix = vectorizer.fit_transform(pr.text[0:10])

# The easiest way to see what happenned is to make a dataframe
results = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names())
results

,"1,105,000","1,160,000",100,1099,"13,381",142nd,15th,170,"170,000","179,550",...,x-ray,yanukovych,year,yearli,yesterday,york,young,younger,yulia,–through
0,0,0,2,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,2,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,1,0,3,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,3,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
6,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,0,0,1,1,0,0,0,1,0,0,...,0,0,4,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,6,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Each row is a document, and each column corresponds to a single vocabulary word or token. And there are a lot of columns, which means we can think of these as points in 1,507 dimensional space.


 

In [257]:
vectorizer.get_feature_names()

['1,105,000',
 '1,160,000',
 '100',
 '1099',
 '13,381',
 '142nd',
 '15th',
 '170',
 '170,000',
 '179,550',
 '1903',
 '1980s',
 '1983',
 '1996',
 '1998',
 '20,000',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '265,950',
 '3.8',
 '315,000',
 '335',
 '350',
 '351,554',
 '4,600',
 '47,200',
 '5,000',
 '51.5m',
 '519',
 '5:00',
 '5million',
 '6,400',
 '6.5',
 '650,000',
 '65m',
 '7.8',
 '750,000',
 '770,000',
 '771',
 '929,088',
 '96-3',
 'able',
 'abuse',
 'access',
 'according',
 'accountability',
 'acquisition',
 'act',
 'action',
 'add',
 'added',
 'additionally',
 'address',
 'addresses',
 'adopted',
 'advance',
 'adversely',
 'advocate',
 'affect',
 'affected',
 'affiliates',
 'afford',
 'afg',
 'afin',
 'african',
 'african-american',
 'african-americans',
 'agencies',
 'agency',
 'agenda',
 'agents',
 'ahmadinejad',
 'aid',
 'air',
 'alarm',
 'ali',
 'allow',
 'allows',
 'ambassador',
 'amendment',
 'amendments',
 'america',
 'american',
 'americans',
 'anniversary',
 'announce',


Look at all those columns with number names! We didn't remove them in the tokenizer, so they're tokens. Do we want that? It depends! Sometimes the numbers in the text can be interesting information.

Now we try this again with tf-idf weighting.

In [333]:
vectorizer = TfidfVectorizer(stop_words='english', tokenizer=tokenize)

matrix = vectorizer.fit_transform(pr.text)

# The easiest way to see what happenned is to make a dataframe
tfidf_vectors = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names())
tfidf_vectors.head()

,'01,'activist,'acts,'disappointing,'e-verify,'em,'liberty,'ll,'no,'re,...,…oil-drilling,…struggling,…tarp,…that,…the,…then,…there,…these,…this,…we
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now let's look at the top words in the press release again. The stop words have gone.

In [338]:
print_sorted_vector(tfidf_vectors.iloc[212,:])


('nti', 0.56487288350283282)
('transit', 0.30992020013592608)
('transportation', 0.23490806362768965)
('training', 0.20020090792648149)
('rutgers', 0.16379220882186274)
('4.3', 0.15701624958670632)
('institute', 0.15030236459784593)
('public', 0.13468667783892418)
('efficient', 0.13247842020300446)
('workers', 0.11513111737966622)
('grant', 0.099558443055257573)
('award', 0.098396096503811856)
('critical', 0.095532368614887939)
('larrousse.nti', 0.091692332362515894)
('premiere', 0.086826433625999941)
('industry', 0.083008913035465287)
('lautenberg', 0.081471107009427737)
('interconnected', 0.080696126214690411)
('element', 0.080696126214690411)
('1991', 0.080696126214690411)


Comapre this to the headline:

In [342]:
doc.sentences[0]

Sentence("Menendez and Lautenberg Applaud USDOT’s $4.3 Million Award to National Transit Institute at Rutgers
                    
                            Funding will provide job training and education for public transportation workers
                    
                      August 3, 2011
                     WASHINGTON, D.C. – Today, U.S.")